### test implementing basic EMF iterations

In [1]:
import numpy as np
import h5py

In [2]:
from sklearn.utils.fixes import expit    
from sklearn.utils.extmath import safe_sparse_dot
from sklearn import linear_model, datasets, metrics, preprocessing 
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
def sig_means(x, b, W):
    a = safe_sparse_dot(x, W.T) + b
    return expit(a, out=a)

### use julia data set

I don't know how to reproduce their normalization yet

In [4]:
hf =  h5py.File('mnist.h5','r')
print('List of arrays in this file: \n', hf.keys())
X = np.array(hf.get('HDF5.name___X'))
Y = np.array(hf.get('HDF5.name___y'))
print X.shape, Y.shape
hf.close()
print "norm of X ",np.linalg.norm(X,ord=2)

('List of arrays in this file: \n', [u'HDF5.name___X', u'HDF5.name___y'])
(60000, 784) (60000,)
norm of X  2117.63422548


In [5]:
from sklearn.externals.six.moves import xrange
from sklearn.utils import check_array
from sklearn.utils import check_random_state
from sklearn.utils import gen_even_slices
from sklearn.utils import issparse
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.utils.extmath import log_logistic
from sklearn.utils.fixes import expit             # logistic function
from sklearn.utils.validation import check_is_fitted

In [6]:
def init_v_bias(X):
    # If the user specifies the training dataset, it can be useful to                                                                                   
    # initialize the visibile biases according to the empirical expected                                                                                
    # feature values of the training data.                                                                                                              
    #                                                                                                                                                   
    # TODO: Generalize this biasing. Currently, the biasing is only written for                                                                         
    #       the case of binary RBMs.
    eps = 1e-8

    probVis = np.mean(X,axis=0)             # Mean across  samples    
    probVis[probVis < eps] = eps            # Some regularization (avoid Inf/NaN)  
    print np.linalg.norm(probVis)

    #probVis[probVis < (1.0-eps)] = (1.0-eps)   
    v_bias = np.log(probVis / (1.0-probVis)) # Biasing as the log-proportion  
    return v_bias

In [7]:
class EMF_RBM():
    def __init__(self, n_components=256, learning_rate=0.005, batch_size=100,
                 n_iter=20, verbose=0, random_state=None, momentum = 0.5, decay = 0.01, weight_decay='L1'):
        self.n_components = n_components
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.n_iter = n_iter
        self.verbose = verbose
        
        self.momentum = momentum
        self.decay = decay
        self.weight_decay = weight_decay
            
        
        # self.random_state_ = random_state
        # always start with new random state
        self.rng = check_random_state(random_state)
        
        # initialize arrays to 0
        self.W = np.asarray(
            self.rng.normal(
                0,
                0.001,
                (self.n_components, X.shape[1])
            ),
            order='fortran')
        
        self.dW_prev = np.zeros_like(self.W)
        self.W2 = self.W*self.W


        self.h_bias = np.zeros(self.n_components, )
        #self.v_bias = np.zeros(X.shape[1], )
        self.h_samples_ = np.zeros((self.batch_size, self.n_components))
        
        # learning rate / mini_batch
        self.lr = 0.0
        
        print "W norm ", np.linalg.norm(self.W, ord=2)
        
        # init vbias
        self.v_bias = init_v_bias(X)
        print "v bias  ",self.v_bias.shape

        print "v bias norm ", np.linalg.norm(self.v_bias, ord=2)



In [8]:
def sample_layer(rbm, layer):
    return (rbm.rng.random_sample(size=l.shape) < layer)  

In [9]:
def _sample_hiddens(rbm, v):
    return sample_layer(rbm, _mean_hiddens(rbm, v))

In [10]:
def _mean_hiddens(rbm, v):
    p = safe_sparse_dot(v, rbm.W.T) + rbm.h_bias
    return expit(p, out=p)

In [11]:
def _mean_visibles(rbm, h):
    p = np.dot(h, rbm.W) + rbm.v_bias
    return expit(p, out=p)

In [12]:
def _sample_visibles(rbm, h):
    return sample_layer(rbm, _mean_visible(rbm, h))

In [13]:
# init starts with samples = [0] :| ?
def init_batch(rbm, vis):
    v_pos = vis
    v_init = v_pos
    
    h_pos = _mean_hiddens(rbm, v_pos)
    h_init = h_pos
   
    return v_pos, h_pos, v_init, h_init

In [14]:
def fit(rbm, X):
    
    n_samples = X.shape[0]
    n_batches = int(np.ceil(float(n_samples) / rbm.batch_size))
    
    print "fitting with n_batches = ",n_batches, " in ",rbm.n_iter, "iterations"
    
    batch_slices = list(gen_even_slices(n_batches * rbm.batch_size,
                                        n_batches, n_samples))
    for iter in xrange(1, rbm.n_iter + 1):
        for batch_slice in batch_slices:
            fit_batch(rbm, X[batch_slice])

        print iter , " . " ,


### run an RBM right here

In [15]:
def equilibrate(rbm, v0, h0, iters=5):
    mv = v0
    mh = h0
    for i in range(iters):
        mv = 0.5 *mv_update(rbm, mv, mh) + 0.5*mv
        mh = 0.5 *mh_update(rbm, mv, mh) + 0.5*mh

    return mv, mh

In [16]:
def mv_update(rbm, v, h):  
    a = np.dot(h, rbm.W) + rbm.v_bias

    h_fluc = h-(h*h)
    a += h_fluc.dot(rbm.W2)*(0.5-v)
    
    return expit(a, out=a)

In [17]:
def mh_update(W2, v, h):
    a = safe_sparse_dot(v, rbm.W.T) + rbm.h_bias
    
    v_fluc = v-(v*v)
    a += v_fluc.dot(rbm.W2.T)*(0.5-h)

    return expit(a, out=a)

In [18]:
def weight_gradient(rbm, v_pos, h_pos ,v_neg, h_neg):
    # naive  / mean field
    dW = safe_sparse_dot(v_pos.T, h_pos, dense_output=True).T - np.dot(h_neg.T, v_neg)
    
    #print "dW naive", np.linalg.norm(dW, ord=2)
    # tap2 correction
    h_fluc = (h_neg - (h_neg*h_neg)).T
    v_fluc = (v_neg - (v_neg*v_neg))
    dW -= h_fluc.dot(v_fluc)*rbm.W

    return dW

In [19]:
def fit_batch(rbm, X_batch):    
 
    lr = float(rbm.learning_rate) / X_batch.shape[0]
    decay = rbm.decay
    
    #print "W, hb vb norm ", np.linalg.norm(rbm.W, ord=2), np.linalg.norm(rbm.h_bias, ord=2), np.linalg.norm(rbm.v_bias, ord=2)
    #print "batch norm ", np.linalg.norm(X_batch, ord=2)
    
    v_pos, h_pos, v_init, h_init = init_batch(rbm, X_batch)
 
    #print "v, h init norm ", np.linalg.norm(v_init, ord=2), np.linalg.norm(h_init, ord=2)
    #print "vb norm "


    # get_negative_samples
    v_neg, h_neg = equilibrate(rbm, v_init, h_init) 
    #print "v, h neg norm ", np.linalg.norm(v_neg, ord=2), np.linalg.norm(h_neg, ord=2)

    # basic gradient
    dW = weight_gradient(rbm, v_pos, h_pos ,v_neg, h_neg) 
    
    #print "dW norm ", np.linalg.norm(dW, ord=2)


    # regularization based on weight decay
    #  similar to momentum >
    if rbm.weight_decay == "L2":
        dW += decay * np.sign(rbm.W)
    elif rbm.weight_decay == "L1":
        dW += decay * rbm.W

    # can we use BLAS here ?
    # momentum
    # note:  what do we do if lr changes per step ?    
    dW = rbm.momentum * rbm.dW_prev  
    rbm.W += lr * dW  
    
    # for next iteration
    rbm.dW_prev =  dW  
    rbm.W2 = rbm.W*rbm.W
    
    # update bias terms
    rbm.h_bias += lr * (h_pos.sum(axis=0) - h_neg.sum(axis=0))
    rbm.v_bias += lr * (np.asarray(v_pos.sum(axis=0)).squeeze() - v_neg.sum(axis=0))

    # only resample (binomial) for CD
    # h_neg[rbm.rng.uniform(size=h_neg.shape) < h_neg] = 1.0  
    # rbm.h_samples_ = np.floor(h_neg, h_neg)
        
    return 0

In [20]:
rbm = EMF_RBM(momentum=0.5, decay=0.01, learning_rate=0.005, n_iter=20)

W norm  0.0438319055922
8.42502854466
v bias   (784,)
v bias norm  195.421791256


In [21]:
fit(rbm, X)

fitting with n_batches =  600  in  20 iterations
1  .  2  .  3  .  4  .  5  .  6  .  7  .  8  .  9  .  10  .  11  .  12  .  13  .  14  .  15  .  16  .  17  .  18  .  19  .  20  . 


### try classifier

#### should we be using the EMF estimator?

what are the correlations...do they drop to 0 as we converge ?

In [23]:
from sklearn import linear_model, datasets, metrics, preprocessing 
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

In [24]:
p = sig_means(X, rbm.h_bias , rbm.W)


In [25]:
print p.shape, Y.shape

(60000, 256) (60000,)


In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(p, Y, test_size=0.2,random_state=0)

In [27]:

for c in [5000]:
    lr  = linear_model.LogisticRegression()
    lr.C = c
    lr.fit(X_train, Y_train)
    Y_test_pred = lr.predict(X_test)
    acc = accuracy_score(Y_test, Y_test_pred)

    print c, acc

5000 0.903833333333


In [28]:
### note bad, but not great